In [21]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import os
import copy

# Define position-specific feature weights
# play around with these weights to see how they affect the model
POSITION_FEATURE_WEIGHTS = {
    'OFF': {
        'finishing': 4,
        'creativity': 2,
        'distribution': 1,
        'defense': 1,
        'duels': 2
    },
    'MID': {
        'finishing': 2,
        'creativity': 3,
        'distribution': 3,
        'defense': 2,
        'duels': 2
    },
    'DEF': {
        'finishing': 1,
        'creativity': 1,
        'distribution': 2,
        'defense': 4,
        'duels': 3
    }
}

def apply_position_weights(df, sheet_name, feature_cols):
    """
    Apply position-specific weights to features based on sheet name.
    
    Args:
        df: DataFrame to modify
        sheet_name: Name of the sheet (used to determine position)
        feature_cols: List of feature columns to weight
    
    Returns:
        Tuple of (weighted DataFrame, list of weighted feature columns)
    """
    # Create a copy to avoid modifying the original dataframe
    df_weighted = df.copy()
    weighted_feature_cols = []
    
    # Determine position from sheet name
    position = None
    if 'off' in sheet_name.lower():
        position = 'OFF'
    elif 'mid' in sheet_name.lower():
        position = 'MID'
    elif 'def' in sheet_name.lower():
        position = 'DEF'
    else:
        print(f"Warning: Could not determine position from sheet name '{sheet_name}'.")
        print("Using equal weights for all features.")
        return df_weighted, feature_cols  # Return original columns if position unknown
    
    print(f"Applying {position} weights to sheet '{sheet_name}'")
    weights = POSITION_FEATURE_WEIGHTS[position]
    
    # Apply weights to each feature column
    for feature in feature_cols:
        if feature in weights and feature in df_weighted.columns:
            feature_weight = weights[feature]
            df_weighted[f'{feature}_weighted'] = df_weighted[feature] * feature_weight
            weighted_feature_cols.append(f'{feature}_weighted')
            print(f"  - Applied weight {feature_weight} to '{feature}'")
    
    # Calculate weighted score
    if weighted_feature_cols:
        df_weighted['weighted_score'] = df_weighted[weighted_feature_cols].sum(axis=1)
        
        # Normalize the weighted score to be between 0 and 1
        min_score = df_weighted['weighted_score'].min()
        max_score = df_weighted['weighted_score'].max()
        if max_score > min_score:
            df_weighted['weighted_score'] = (df_weighted['weighted_score'] - min_score) / (max_score - min_score)
        
        print(f"  - Created 'weighted_score' column")
    
    return df_weighted, weighted_feature_cols

def train_and_evaluate_sheet(df, sheet_name, feature_cols, target_fifa, target_real, use_weighted_features=True):
    """
    Train and evaluate Random Forest models on a single sheet's data.
    
    Args:
        df: DataFrame containing the sheet data
        sheet_name: Name of the sheet being processed
        feature_cols: List of feature columns to use
        target_fifa: Column name for FIFA rating target
        target_real: Column name for real-life rating target
        use_weighted_features: Whether to apply position-based weights
        
    Returns:
        Dictionary containing models and evaluation metrics
    """
    print(f"\n===== Processing Sheet: {sheet_name} =====")
    
    # Check if required columns exist
    missing_features = [col for col in feature_cols if col not in df.columns]
    if missing_features:
        print(f"Warning: Missing features in {sheet_name}: {missing_features}")
        available_features = [col for col in feature_cols if col in df.columns]
        if not available_features:
            print(f"Error: No valid features available in {sheet_name}. Skipping.")
            return None
        print(f"Proceeding with available features: {available_features}")
        feature_cols = available_features
    
    if target_fifa not in df.columns:
        print(f"Error: Target column '{target_fifa}' not found in {sheet_name}. Skipping FIFA model.")
        has_fifa_target = False
    else:
        has_fifa_target = True
        
    if target_real not in df.columns:
        print(f"Error: Target column '{target_real}' not found in {sheet_name}. Skipping Real model.")
        has_real_target = False
    else:
        has_real_target = True
        
    if not has_fifa_target and not has_real_target:
        print(f"Error: No target columns found in {sheet_name}. Skipping sheet.")
        return None
    
    # Apply position-specific weights if requested
    if use_weighted_features:
        df_weighted, weighted_feature_cols = apply_position_weights(df, sheet_name, feature_cols)
        
        # Add the weighted score as an additional feature
        #if 'weighted_score' in df_weighted.columns:
            #weighted_feature_cols.append('weighted_score')
            
        # Use weighted features if available, otherwise use original features
        if weighted_feature_cols:
            X = df_weighted[weighted_feature_cols]
            print(f"Using weighted features: {weighted_feature_cols}")
        else:
            X = df[feature_cols]
            print(f"Using original features: {feature_cols}")
            
        # For the targets, still use the original dataframe
        features_used = weighted_feature_cols
    else:
        # Use original features without weighting
        X = df[feature_cols]
        features_used = feature_cols
    
    results = {
        'sheet_name': sheet_name,
        'feature_cols': features_used,
    }
    
    # Process FIFA target if available
    if has_fifa_target:
        y_fifa = df[target_fifa]
        
        # Split data
        X_train, X_test, y_fifa_train, y_fifa_test = train_test_split(
            X, y_fifa, test_size=0.2, random_state=42
        )
        
        # Train model
        fifa_model = RandomForestRegressor(random_state=42)
        fifa_model.fit(X_train, y_fifa_train)
        
        # Make predictions
        fifa_preds = fifa_model.predict(X_test)
        
        # Calculate metrics
        fifa_r2 = r2_score(y_fifa_test, fifa_preds)
        fifa_rmse = np.sqrt(mean_squared_error(y_fifa_test, fifa_preds))
        
        # Print results
        print(f"--- {sheet_name} - FIFA Rating Prediction ---")
        print(f"R²: {fifa_r2:.3f}")
        print(f"RMSE: {fifa_rmse:.3f}")
        
        # Plot feature importance
        plt.figure(figsize=(10, 6))
        plot_importance(fifa_model, features_used, f"Feature Importance - FIFA Rating ({sheet_name})")
        plt.savefig(f"FIFA_importance_{sheet_name.replace(' ', '_')}.png")
        plt.close()
        
        # Store results
        results['fifa_model'] = fifa_model
        results['fifa_r2'] = fifa_r2
        results['fifa_rmse'] = fifa_rmse
        results['fifa_importances'] = fifa_model.feature_importances_
    
    # Process Real-Life target if available
    if has_real_target:
        y_real = df[target_real]
        
        # Split data
        X_train, X_test, y_real_train, y_real_test = train_test_split(
            X, y_real, test_size=0.2, random_state=42
        )
        
        # Train model
        real_model = RandomForestRegressor(random_state=42)
        real_model.fit(X_train, y_real_train)
        
        # Make predictions
        real_preds = real_model.predict(X_test)
        
        # Calculate metrics
        real_r2 = r2_score(y_real_test, real_preds)
        real_rmse = np.sqrt(mean_squared_error(y_real_test, real_preds))
        
        # Print results
        print(f"--- {sheet_name} - Real-Life Rating Prediction ---")
        print(f"R²: {real_r2:.3f}")
        print(f"RMSE: {real_rmse:.3f}")
        
        # Plot feature importance
        plt.figure(figsize=(10, 6))
        plot_importance(real_model, features_used, f"Feature Importance - Real-Life Rating ({sheet_name})")
        plt.savefig(f"Real_importance_{sheet_name.replace(' ', '_')}.png")
        plt.close()
        
        # Store results
        results['real_model'] = real_model
        results['real_r2'] = real_r2
        results['real_rmse'] = real_rmse
        results['real_importances'] = real_model.feature_importances_
    
    return results

def plot_importance(model, feature_names, title):
    """Plot feature importance for a model"""
    importances = model.feature_importances_
    indices = np.argsort(importances)
    
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.title(title)
    plt.xlabel("Importance")
    plt.tight_layout()

def compare_sheets_results(all_results):
    """Create comparison plots and tables for all sheets"""
    if not all_results:
        print("No valid results to compare")
        return
    
    # Compare FIFA R² across sheets
    sheets = [r['sheet_name'] for r in all_results if 'fifa_r2' in r]
    if sheets:
        fifa_r2_values = [r['fifa_r2'] for r in all_results if 'fifa_r2' in r]
        
        plt.figure(figsize=(10, 6))
        plt.bar(sheets, fifa_r2_values)
        plt.title("FIFA Rating Model - R² by Sheet")
        plt.ylabel("R²")
        plt.ylim(0, 1)  # R² is typically between 0 and 1
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig("FIFA_R2_comparison.png")
        plt.close()
    
    # Compare Real-Life R² across sheets
    sheets = [r['sheet_name'] for r in all_results if 'real_r2' in r]
    if sheets:
        real_r2_values = [r['real_r2'] for r in all_results if 'real_r2' in r]
        
        plt.figure(figsize=(10, 6))
        plt.bar(sheets, real_r2_values)
        plt.title("Real-Life Rating Model - R² by Sheet")
        plt.ylabel("R²")
        plt.ylim(0, 1)  # R² is typically between 0 and 1
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig("Real_R2_comparison.png")
        plt.close()
    
    # Create summary table
    summary_data = []
    for result in all_results:
        sheet_name = result['sheet_name']
        row = {'Sheet': sheet_name}
        
        if 'fifa_r2' in result:
            row['FIFA R²'] = f"{result['fifa_r2']:.3f}"
            row['FIFA RMSE'] = f"{result['fifa_rmse']:.3f}"
        else:
            row['FIFA R²'] = "N/A"
            row['FIFA RMSE'] = "N/A"
            
        if 'real_r2' in result:
            row['Real R²'] = f"{result['real_r2']:.3f}"
            row['Real RMSE'] = f"{result['real_rmse']:.3f}"
        else:
            row['Real R²'] = "N/A"
            row['Real RMSE'] = "N/A"
            
        summary_data.append(row)
    
    summary_df = pd.DataFrame(summary_data)
    print("\n===== Summary of Results =====")
    print(summary_df)
    
    # Save summary to CSV
    summary_df.to_csv("model_performance_summary.csv", index=False)
    print("Summary saved to 'model_performance_summary.csv'")

def process_all_sheets(file_path, feature_cols, target_fifa, target_real, use_weighted_features=True):
    """
    Process all sheets in an Excel file with the RF model
    
    Args:
        file_path: Path to the Excel file
        feature_cols: List of feature columns to use
        target_fifa: Column name for FIFA rating target
        target_real: Column name for real-life rating target
        use_weighted_features: Whether to apply position-based weights
    """
    print(f"Processing file: {file_path}")
    
    # Load all sheets
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    print(f"Found {len(all_sheets)} sheets: {list(all_sheets.keys())}")
    
    # Process each sheet
    all_results = []
    
    for sheet_name, df in all_sheets.items():
        if df.empty:
            print(f"Sheet '{sheet_name}' is empty. Skipping.")
            continue
            
        result = train_and_evaluate_sheet(df, sheet_name, feature_cols, target_fifa, target_real, use_weighted_features)
        if result:
            all_results.append(result)
    
    # Compare results across sheets
    compare_sheets_results(all_results)
    
    return all_results


In [22]:

# --- Constants ---
FEATURE_COLS = ['finishing', 'creativity', 'distribution', 'defense', 'duels']
TARGET_FIFA = 'Fifa Ability Overall'  
TARGET_REAL = 'Rating'

# File to process - change to your normalized or weighted file
FILE_PATH = '../notebooks/Composite_Features_Output_normalized.xlsx'  # or 'General_Weighted_Features.xlsx'
# Whether to apply position-specific weights
USE_WEIGHTED_FEATURES = True

# Process all sheets
results = process_all_sheets(FILE_PATH, FEATURE_COLS, TARGET_FIFA, TARGET_REAL, USE_WEIGHTED_FEATURES)

print("\nProcessing complete!")
print(f"Model performance summary and feature importance plots have been saved.")

# Optional: Save position-weighted data to a new Excel file
if USE_WEIGHTED_FEATURES:
    print("\nCreating position-weighted Excel file...")
    all_sheets = pd.read_excel(FILE_PATH, sheet_name=None)
    
    with pd.ExcelWriter('Position_Weighted_Features.xlsx') as writer:
        for sheet_name, df in all_sheets.items():
            if df.empty:
                continue
            
            weighted_df, _ = apply_position_weights(df, sheet_name, FEATURE_COLS)
            weighted_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print("Position-weighted data saved to 'Position_Weighted_Features.xlsx'")

Processing file: ../notebooks/Composite_Features_Output_normalized.xlsx
Found 4 sheets: ['Data', 'DEF', 'MID', 'OFF']

===== Processing Sheet: Data =====
Using equal weights for all features.
Using weighted features: ['finishing', 'creativity', 'distribution', 'defense', 'duels']
--- Data - FIFA Rating Prediction ---
R²: 0.219
RMSE: 4.684
--- Data - Real-Life Rating Prediction ---
R²: 0.652
RMSE: 0.169

===== Processing Sheet: DEF =====
Applying DEF weights to sheet 'DEF'
  - Applied weight 1 to 'finishing'
  - Applied weight 1 to 'creativity'
  - Applied weight 2 to 'distribution'
  - Applied weight 4 to 'defense'
  - Applied weight 3 to 'duels'
  - Created 'weighted_score' column
Using weighted features: ['finishing_weighted', 'creativity_weighted', 'distribution_weighted', 'defense_weighted', 'duels_weighted']
--- DEF - FIFA Rating Prediction ---
R²: 0.126
RMSE: 4.990
--- DEF - Real-Life Rating Prediction ---
R²: 0.551
RMSE: 0.174

===== Processing Sheet: MID =====
Applying MID weig